# Deep Knowledge Tracing

This notebook will show you how to train and use the DKT.
First, we will show how to get the data (here we use a0910 as the dataset).
Then we will show how to train a DKT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [DKT.py](DKT.py)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
from XKT.DKT import etl
batch_size = 32
train = etl("../../data/a0910c/train.json", batch_size=batch_size)
valid = etl("../../data/a0910c/valid.json", batch_size=batch_size)
test = etl("../../data/a0910c/test.json", batch_size=batch_size)

reading data from ../../data/a0910c/train.json: 3966it [00:00, 23302.50it/s]
batchify: 100%|██████████| 130/130 [00:00<00:00, 629.18it/s]
reading data from ../../data/a0910c/valid.json: 472it [00:00, 27564.52it/s]
e:\program\baize\baize\extlib\bucketing.py:327: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[55, 58, 59, 61, 65, 69, 74, 76, 77, 79, 80, 88, 90, 94, 95, 96, 99]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
batchify: 100%|██████████| 84/84 [00:00<00:00, 1241.57it/s]
reading data from ../../data/a0910c/test.json: 1088it [00:00, 13857.38it/s]
e:\program\baize\baize\extlib\bucketing.py:327: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[73, 88]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
batchify: 100%|██████████| 101/101 [00:00<00:00, 931.51it/s]


## Training and Persistence

In [2]:
from XKT import DKT
model = DKT(hyper_params=dict(ku_num=146, hidden_num=100))
model.train(train, valid, end_epoch=2)
model.save("dkt")

logger: <Logger model (INFO)>
model_name: model
model_dir: model
begin_epoch: 0
end_epoch: 2
batch_size: 32
save_epoch: 1
optimizer: Adam
optimizer_params: {'learning_rate': 0.001, 'wd': 0.0001, 'clip_gradient': 1}
lr_params: {}
train_select: None
save_select: None
ctx: cpu(0)
train_ctx: None
eval_ctx: None
toolbox_params: {}
hyper_params: {'ku_num': 146, 'hidden_num': 100}
init_params: {}
loss_params: {}
caption: 
validation_result_file: model\result.json
cfg_path: model\configuration.json
Epoch| Total-E          Batch     Total-B       Loss-SequenceLogisticMaskLoss             Progress           
    0|       1            130         130                            0.639498     [00:04<00:00, 30.63it/s]   
Epoch [0]	Loss - SequenceLogisticMaskLoss: 0.639498
           precision    recall        f1  support
0.0         0.427845  0.205795  0.277913     7765
1.0         0.689022  0.864755  0.766951    15801
macro_avg   0.558433  0.535275  0.522432    23566
accuracy: 0.647628	macro_auc: 0.

evaluating: 100%|██████████| 84/84 [00:00<00:00, 130.78it/s]
model, INFO writing configuration parameters to G:\program\XKT\examples\DKT\dkt\configuration.json


'dkt'

## Loading and Testing

In [3]:
model = DKT.from_pretrained("dkt")
print(model.eval(test))


evaluating: 100%|██████████| 101/101 [00:00<00:00, 113.98it/s]


           precision    recall        f1  support
0.0         0.484619  0.157390  0.237611    17517
1.0         0.670330  0.911000  0.772351    32944
macro_avg   0.577475  0.534195  0.504981    50461
accuracy: 0.649393	macro_auc: 0.570926	macro_aupoc: 0.702939


## Predict

In [ ]:
import mxnet as mx
inputs = mx.nd.ones((2, 3))  # (2 students, 3 steps)
outputs, _ = model(inputs)
outputs.shape